# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

- Intermediate-level knowledge of Python 3 (NumPy and Pandas preferably, but not required)
- Exposure to PyTorch usage
- Basic understanding of Deep Learning and Language Models (BERT specifically)

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.png" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('Data/smile-annotations-final.csv', 
                 names=['id', 'text','category'])
df.set_index('id', inplace=True)

In [3]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [4]:
df.text.iloc[0]

'@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap'

In [5]:
df.category.value_counts()
#nocode is simply no clear emotions in this tweet

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [6]:
#we want to remove nocode and also those with multiple emotions, with |
df = df[~df.category.str.contains('\|')]   #we need to choose the | by backslash
df = df[df.category != 'nocode']

In [7]:
df.category.value_counts()
#class imbalance

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [8]:
#build dictionary, key: emotion, value: numbers
possible_labels = df.category.unique()

In [9]:
label_dict = {}
#loop over index
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [10]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [11]:
#build new column for these values
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


## Task 3: Training/Validation Split

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
#stratified split
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                 df.label.values,
                                                 test_size = 0.15,
                                                 random_state=17,
                                                 stratify = df.label.values
                                                 )

In [14]:
df['data_type'] = ['not_set']*df.shape[0]

In [15]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [16]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [17]:
df.groupby(['category', 'label', 'data_type']).count()
#group by using count

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Task 4: Loading Tokenizer and Encoding our Data

In [18]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         #all lower case
                                         do_lower_case = True,
                                         )

In [20]:
#batch using multiple strings and convert them into tokens
encoded_data_train = tokenizer.batch_encode_plus(
        df[df.data_type == 'train'].text.values,
        add_special_tokens = True,
        #to know when sentence begins and ends
        return_attention_mask = True,
        #set max length to large values for big sentences
        pad_to_max_length = True,
        max_length = 256,
        return_tensors = 'pt'
        #pt: pytorch
        )

encoded_data_val = tokenizer.batch_encode_plus(
        df[df.data_type == 'val'].text.values,
        add_special_tokens = True,
        #to know when sentence begins and ends
        return_attention_mask = True,
        #set max length to large values for big sentences
        pad_to_max_length = True,
        max_length = 256,
        return_tensors = 'pt'
        #pt: pytorch
        )

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

In [21]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                              attention_masks_val,
                              labels_val)

In [22]:
len(dataset_train)

1258

In [23]:
len(dataset_val)

223

## Task 5: Setting up BERT Pretrained Model

In [24]:
from transformers import BertForSequenceClassification

In [25]:
#each sequence will be dealt separate classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    #the other cased one is larger and takes more computation power
    #we want to fine tune the parts we need
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
                                     )
#450 MB needs to be fetched and loaded into memory
#bert takes into text and encodes into meaningful way according to the huge corpus it was intitially exposed to
#we are just lying on top of it to get our 6 classes classifier

## Task 6: Creating Data Loaders

In [26]:
from torch.utils.data import DataLoader,
                            RandomSampler,     #will use for training
                            SequentialSampler  
                    #will use for our validation dataset, gradients are fixed

IndentationError: unexpected indent (<ipython-input-26-b78e43e8bd47>, line 2)

In [ ]:
batch_size = 4   #very small due to machine low specs but can increase to 32

dataloader_train = DataLoader(
            dataset_train,
            sampler = RandomSampler(dataset_train),
            #to avoid it learning from any sequences
            batch_size = batch_size
            )

dataloader_val = DataLoader(
            dataset_val,
            sampler = RandomSampler(dataset_val),
            #to avoid it learning from any sequences
            batch_size = 32    #here no many computation, no backpropagation
            )

## Task 7: Setting Up Optimizer and Scheduler

In [ ]:
#Optimizer defines our learning rate and how it changed throught each epoch
from transformers import AdamW, get_linear_schedule_with_warmup
#Adam with weight decay, stochastic optimizer

In [ ]:
optimizer = AdamW(
                model.parameters(),
                lr = 1e-5,         #recommended: 2e-5 > 5e-5
                eps = 1e-8,
                )

In [ ]:
epochs = 10

schedular = get_linear_schedule_with_warmup(
        optimizer,     #Adam
        num_warmup_steps = 0,
        num_training_steps = len(dataloader_train)*epochs
        )

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [27]:
import numpy as np

In [28]:
from sklearn.metrics import f1_score

preds = [0.9 0.05 0.05 0 0 0]

we want to convert it to [1 0 0 0 0 0]

In [29]:
#f1-score is good bec. of class imbalance
#accuracy alone will give me skewed results,
    #based on f1-score not actually representing what we want

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    #flatten to get single list and not array
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')
#can changed weighted to macro

In [30]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(label_flat):
        y_preds = pred_flat[labels_flat == label]
#here we are using numpy indexing to index 2 array of the same shape by each other
        y_true = labels_flat[labels_flat == label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:  {len(y_preds[y_preds == label])}/{len(y_true)}\n')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [31]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#send model to device we are using
model.to(device)
print(device)

cpu


In [33]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [35]:
for epoch in tqdm(range(1, epoch+1)):
    
    model.train()
    
    loss_train_total = 0
    #we set it initially as 0
    
    progress_bar = tqdm(dataloader_train,
                        desc = 'Epoch {:1d}'.format(epoch),
                        leave = False,   #overwrite after each epoch
                        disable = False                        
                       )
    #to see where are we, has it crashed
    
    for batch in progress_bar:
        model.zero_grad()
        #gradient set to zero
        
        batch = tuple(b.to(device) for b in batch)
        #this is imp for cuda gpu use
        
        inputs = {
            'input_ids':         batch[0],
            'attention_mask':    batch[1],
            'labels' :           batch[2]
        }
        
        outputs = model(**inputs)
        #outputs dictionary directly into inputs
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        #clip our gradient
        #take gradient and give it normal value that we provide as 1
        #stop gradients from slipping into becoming exceptionally small or too big
        #promote generalization
        
        optimizer.step()
        schedular.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        #append small dictionary
        
        torch.save(model.state_dict(), f'Models/Bert_ft_epoch{epoch}.model')
        
        tqdm.write('\nEpoch {epoch}')
        
        loss_train_avg = loss_train_total/len(dataloader)
        tqdm.write('Training loss: {loss_train_avg}')
        
        val_loss, predictions, true_vals = evaluate(dataloader_val)
        #this is imp if over training
        #model will have no generalization abilities
        val_f1 = f1_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (weighted): {val_f1}')
        
#Cpu takes 40 minutes
#gpu takes 30 seconds

NameError: name 'epoch' is not defined

## Task 10: Loading and Evaluating our Model

In [78]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [79]:
model.to(device)
pass   #to not get alot of text output

In [80]:
model.load_state_dict(
    torch.load('Models/finetuned_bert_epoch_1_gpu_trained.model',
              map_location = torch.device('cpu')))

<All keys matched successfully>

In [81]:
_, prediction, true_vals = evaluate(dataloader_val)
#7 batches
#will take almost 2 minutes

In [82]:
accuracy_per_class(prediction, true_vals)

NameError: name 'label_flat' is not defined

In [83]:
#To summarize:
# model was trained on Google colab --GPU Instance(k80)
# batch size  = 32
# epoch = 10